# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [4]:
import twitter

CONSUMER_KEY = "03Ai0vQvQ5cbkf2PqMLDKqhmE"
CONSUMER_SECRET = "ROGs6elMuhmnfTuky25EhoNvvTbRT9uXe7c4r2GEw6LRQpxopE"

ACCESS_TOKEN_KEY = "4310324793-f8UuXkQui6m0bpwlt9Us3O8LQQMUVebBF8KCs6F"
ACCESS_TOKEN_SECRET = "oySIWDx2F55bTyVNEqvhi5Qmc2ZQUlKtxVrwthB9IQrkQ"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET,
                  sleep_on_rate_limit=True)

Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя. По каждому пользователю достаточно собрать 200 твитов.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. Предполагается, что информация о пользователе содержится в твитах, которые пользователь написал сам. Это означает, что можно попробовать отфильтровать ответы другим пользователям, ссылки и ретвиты, а так же картинки и видео, так как наша цель - найти текстовую информацию.

In [5]:
import re
"""import os
import json
file_name = 'downloded_tweets'
if not os.path.exists(file_name):
    os.makedirs(file_name)"""
    
    
def get_user_tweets(user_id):
    """returns list of tweets as dicts"""
    # your code here
    #global file_name
    user_tweets = []
    try:
        statuses = api.GetUserTimeline(user_id=user_id, count=200, include_rts ='false', exclude_replies ='true') # exclude retweets and replies
    except twitter.TwitterError:
        print 'error'
        statuses = ''
    except ValueError:
        print 'error'
        statuses = ''
    for s in statuses:
        if (not s.urls) and (s.media is None): # do not include url and media (photo and video)
            s.text = re.sub(r"http\S+", "", s.text) # if there is other links in tweets
            user_tweets.append({'lang': s.lang, 
                 'favorited': s.favorited, 
                 'truncated': s.truncated, 
                 'text': s.text, 
                 'created_at': s.created_at, 
                 'retweeted': s.retweeted, 
                 'source': s.source, 
                 'user': {'id': s.user.id}, 
                 'id': s.id})
    return user_tweets

In [ ]:
#get_user_tweets(2330149164)

## Разбор текста твита

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). Обратите внимание, что нужно учесть возможное наличие пунктуации и выделить по возможности только слова. 

In [6]:
def get_words(text):
    """returns list of words"""
    # your code here
    text = text.lower()
    words = []
    for w in text.split():
        w = re.sub(r'[!\?@.,;:\-%\#\(\)\$\\/\'\"\*\+\[\]<>_&\d]', '', w) # delete symbols
        if w: # if w is not empty string
            words.append(w)
    return words

In [ ]:
#get_words(u'😏 ')

Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

In [7]:
import nltk
#nltk.download()

Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [8]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def get_tokens(words):
    """returns list of tokens"""
    # your code here
    tokens = []
    stop_words = stopwords.words('english') # get stop words
    wnl = WordNetLemmatizer()
    for w in words:
        w = wnl.lemmatize(w) # normalize
        if (w not in stop_words) and w.isalpha(): # exclude smiles
            tokens.append(w)
    return tokens

Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита. 

In [9]:
def get_tweet_tokens(tweet):
    # your code here
    words = get_words(tweet)
    return get_tokens(words)

Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [10]:
def count_tokens(user_tokens): # create dictionary to count user tokens
    tokens_amt = {}
    for token in user_tokens:
        if token in tokens_amt:
            tokens_amt[token] += 1
        else:
            tokens_amt[token] = 1
    return tokens_amt

In [11]:
def collect_users_tokens(df_users):
    """returns users list and list of user dicts. Each dict contains frequence of user tokens"""
    # your code here
    # save intermediate result
    users = df_users['uid'].values.tolist()
    user_tokens = [] # list of user tokens
    user_tokens_amt = [] # list of dictionaries
    
    for user in users:
        #print user
        if users.index(user) & 127 == 0:
            print users.index(user)
        user_tweets = get_user_tweets(user) # get user tweets
        for tweet in user_tweets:
            user_tokens += get_tweet_tokens(tweet['text'])
        user_tokens_amt.append(count_tokens(user_tokens))
         
    return users, user_tokens_amt

In [ ]:
import pandas as pd
from collections import defaultdict
import json
from sklearn.feature_extraction import DictVectorizer

"""v = DictVectorizer()
D = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}]
print v.fit_transform(D).toarray()
print v.get_feature_names()"""

TRAINING_SET_URL = "train.csv"
EXAMPLE_SET_URL = "test.csv"
df_users_train = pd.read_csv(TRAINING_SET_URL, sep=",", header=0)
df_users_ex = pd.read_csv(EXAMPLE_SET_URL, sep=",", header=0)
df_users_ex['cls'] = None
df_users = pd.concat([df_users_train, df_users_ex]) # поставить комменнтарий перед строчкой!
#df_users = df_users[3072:3200]


users, users_tokens = collect_users_tokens(df_users) # поставить комменнтарий перед строчкой!

0
error
128
error
error
256
error
384
error
error
error
error
error
error
512
error
error
error
640
error
error
768
error
error
error
896
error
error
error
error
1024
error
error
error
error
error
1152
error
error
error
error
error
1280
error
error
error
error
error
error
1408
error
error
error
1536
error
error
error
1664
error


In [ ]:
df_users3000 = df_users[:3000]
users3000, users_tokens3000 = collect_users_tokens(df_users3000)
print 'ok'
df_users6000 = df_users[3000:6000]
users6000, users_tokens6000 = collect_users_tokens(df_users6000)
print 'ok'
df_users9000 = df_users[6000:9000]
users9000, users_tokens9000 = collect_users_tokens(df_users9000)
print 'ok'
df_users11000 = df_users[9000:]
users11000, users_tokens11000 = collect_users_tokens(df_users11000)
print 'ok'

In [ ]:
import numpy as np
np.savez("files/out_4.dat", data=vs, users=users, users_tokens=users_tokens )

Далее для получения представления о полученной информацию о токенах предлагается отобразить ее в виде облака тэгов. [Подсказка](http://anokhin.github.io/img/tag_cloud.png). 

In [ ]:
v = DictVectorizer()
vs = v.fit_transform(users_tokens)

In [ ]:
def draw_tag_cloud(v, vs):
    """Draws tag cloud of found tokens"""
    # your code here
    return